In [10]:
# 임포트
import json
import time
import random
import requests
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

In [2]:
# 깃에서 파일 불러오기
cortar = pd.read_csv('C:\\oasis\\oasisbusiness-ds\\oasisbusiness-ds\\1_데이터수집\\법정동코드전체.csv')
cortar = cortar[cortar['폐지여부\r\n']=='존재\r\n'] # 존재하는 지역만 남기기
ls_cortar = cortar.drop_duplicates('법정동코드')['법정동코드'] # 법정동 코드만 담기 (총 20551개)

In [11]:
### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

In [ ]:
fails = []
info_df = pd.DataFrame()

# 20551
for i in range(0,5000):
    cortar_url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
    count = 0
    
    for num in range(1,4):
        
        try:
            count += 1
            start = time.time() 
            url = cortar_url + str(num)
            headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
            time.sleep(random.randrange(3,7))
            response = requests.get(url, headers = headers)
            print(response,str(int(time.time()-start)) + 'ms', str(count) +'/3')
            items = response.json()['body']
            df = pd.DataFrame(items)
            info_df = info_df.append(df)
        except:
            fails.append([i,num])
            
    if i % 100 == 0:
        info_df.to_csv('./네이버부동산_'+str(i+1)+'.csv',index=False,encoding='utf-8-sig')

info_df.to_csv('./네이버부동산_'+str(i+1)+'.csv',index=False,encoding='utf-8-sig')        

<Response [307]> 3ms 1/3
<Response [307]> 4ms 2/3
<Response [307]> 4ms 3/3
<Response [307]> 5ms 1/3
<Response [307]> 3ms 2/3
<Response [307]> 6ms 3/3
<Response [307]> 6ms 1/3
<Response [307]> 3ms 2/3
<Response [307]> 4ms 3/3
<Response [307]> 6ms 1/3
<Response [307]> 5ms 2/3
<Response [307]> 3ms 3/3
<Response [307]> 3ms 1/3
<Response [307]> 4ms 2/3
<Response [307]> 6ms 3/3
<Response [307]> 5ms 1/3
<Response [307]> 6ms 2/3
<Response [307]> 6ms 3/3
<Response [307]> 3ms 1/3
<Response [307]> 6ms 2/3
<Response [307]> 5ms 3/3
<Response [307]> 3ms 1/3
<Response [307]> 3ms 2/3
<Response [307]> 4ms 3/3
<Response [307]> 4ms 1/3
<Response [307]> 5ms 2/3
<Response [307]> 6ms 3/3
<Response [307]> 6ms 1/3
<Response [307]> 5ms 2/3
<Response [307]> 3ms 3/3
<Response [200]> 6ms 1/3
<Response [200]> 6ms 2/3
<Response [200]> 6ms 3/3
<Response [200]> 6ms 1/3
<Response [200]> 6ms 2/3
<Response [200]> 5ms 3/3
<Response [200]> 6ms 1/3
<Response [200]> 3ms 2/3
<Response [200]> 5ms 3/3
<Response [200]> 5ms 1/3


In [8]:
# 15601
fails = []
info_df = pd.DataFrame()

cortar_url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(ls_cortar[2])
headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
response = requests.get(cortar_url, headers = headers)

items = response.json()['body']
items

[{'atclNo': '2233528680',
  'cortarNo': '1111010100',
  'atclNm': '일반상가',
  'atclStatCd': 'R0',
  'rletTpCd': 'D02',
  'uprRletTpCd': 'D02',
  'rletTpNm': '상가',
  'tradTpCd': 'B2',
  'tradTpNm': '월세',
  'vrfcTpCd': 'DOC',
  'flrInfo': '1/4',
  'prc': 10000,
  'rentPrc': 420,
  'hanPrc': '1억',
  'spc1': 90,
  'spc2': 90,
  'direction': '남향',
  'atclCfmYmd': '22.09.27.',
  'lat': 37.585205,
  'lng': 126.970532,
  'atclFetrDesc': '청운동 대로변 코너 신축건물 1층상가',
  'tagList': ['2년이내', '융자금없는', '1층', '주차가능'],
  'bildNm': '',
  'minute': 0,
  'sameAddrCnt': 1,
  'sameAddrDirectCnt': 0,
  'cpid': 'NEONET',
  'cpNm': '부동산뱅크',
  'cpCnt': 1,
  'rltrNm': '서촌상가공인중개사',
  'directTradYn': 'N',
  'minMviFee': 0,
  'maxMviFee': 0,
  'etRoomCnt': 0,
  'tradePriceHan': '',
  'tradeRentPrice': 0,
  'tradeCheckedByOwner': False,
  'cpLinkVO': {'cpId': 'NEONET',
   'mobileArticleLinkTypeCode': 'NONE',
   'mobileBmsInspectPassYn': 'Y',
   'pcArticleLinkUseAtArticleTitle': False,
   'pcArticleLinkUseAtCpName': False,
